In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tqdm.keras import TqdmCallback

In [2]:
base_dir = 'Dataset'

In [3]:
# ImageDataGenerator for rescaling (no augmentation for faster training)
train_datagen = ImageDataGenerator(
    rescale=1.0/255,  # Only rescale to reduce computational overhead
    validation_split=0.2  # 20% of the data will be used for validation
)

# Load training and validation data
train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(64, 64),  # Reduced image size for faster computation
    batch_size=16,  # Smaller batch size for faster CPU processing
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(64, 64),
    batch_size=16,  # Keep validation batch size smaller too
    class_mode='categorical',
    subset='validation'
)

Found 16516 images belonging to 15 classes.
Found 4122 images belonging to 15 classes.


In [4]:
#  Build a smaller and faster CNN model
model = Sequential()

# First convolutional layer (fewer filters, smaller image size)
model.add(Conv2D(16, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Second convolutional layer (fewer filters)
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the layers
model.add(Flatten())

# Fully connected layer
model.add(Dense(64, activation='relu'))  # Reduced dense layer size
model.add(Dropout(0.5))  # Dropout for regularization
model.add(Dense(train_generator.num_classes, activation='softmax'))  # Output layer

/Users/maharaja/pytorch-env/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model with tqdm progress bar
history = model.fit(
    train_generator,
    epochs=2,  # 10 epochs as requested
    validation_data=validation_generator,
    callbacks=[TqdmCallback(verbose=1)]  # Use tqdm to track progress
)

# Save the model
model.save('plant_disease_classifier_fast_cnn.h5')

0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

Epoch 1/2


/Users/maharaja/pytorch-env/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
2024-09-24 02:49:34.675279: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:3: Filling up shuffle buffer (this may take a while): 1 of 8
2024-09-24 02:50:05.256290: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:3: Filling up shuffle buffer (this may take a while): 2 of 8
2024-09-24 02:50:34.966467: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:3: Filling up shuffle buffer (this may take a while): 3 of 8
2024-09-24 02:50:54.637593: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:3: Filling up sh

   1/1033 ━━━━━━━━━━━━━━━━━━━━ 51:15:08 179s/step - accuracy: 0.0625 - loss: 2.8289

2024-09-24 02:52:09.563735: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


1033/1033 ━━━━━━━━━━━━━━━━━━━━ 16147s 15s/step - accuracy: 0.2784 - loss: 2.2078 - val_accuracy: 0.6497 - val_loss: 1.1674
Epoch 2/2
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 18s 17ms/step - accuracy: 0.5246 - loss: 1.4327 - val_accuracy: 0.7278 - val_loss: 0.9246


In [11]:
# Evaluate the model on validation data
val_loss, val_acc = model.evaluate(validation_generator)
print(f'Validation Accuracy: {val_acc}')

258/258 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.7192 - loss: 0.9376
Validation Accuracy: 0.727802038192749


In [12]:
import os
import warnings
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import f1_score

# Suppress TensorFlow and Keras logs
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Suppress specific Keras warnings
warnings.filterwarnings("ignore", category=UserWarning, module='keras.src.trainers.data_adapters.py_dataset_adapter')

# Load the saved model and compile it
model = load_model('plant_disease_classifier_fast_cnn.h5')
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Assuming the test dataset is organized in a directory
test_datagen = ImageDataGenerator(rescale=1.0/255)

test_generator = test_datagen.flow_from_directory(
    base_dir,  # Replace with your actual test dataset directory
    target_size=(64, 64),
    batch_size=16,
    class_mode='categorical',
    shuffle=False
)

# Predict the output for test data
y_pred = model.predict(test_generator, steps=test_generator.samples // test_generator.batch_size + 1)

# Convert predicted probabilities to class labels
y_pred_classes = np.argmax(y_pred, axis=1)

# Get true labels from test_generator
y_true = test_generator.classes

# Calculate the F1 score
f1 = f1_score(y_true, y_pred_classes, average='weighted')

print(f"F1 Score: {f1}")

Found 20638 images belonging to 15 classes.
1290/1290 ━━━━━━━━━━━━━━━━━━━━ 18s 14ms/step
F1 Score: 0.7112471774948349
